In [1]:
from tqdm import tnrange, tqdm_notebook
import collections
import math
import numpy as np
from collections import defaultdict
import threading
import functions as f
import networkx as nx
import pandas as pd

#### Creating Graph

<span style="font-family: Babas; font-size: 1.2em;">

In a first time, we want to create the graph. In order to do so, we implemented the function "graph_dict" that returns the graph as a dictionary structure

</span>

In [42]:
graph = f.graph_dict()

<span style="font-family: Babas; font-size: 1.2em;">

And now we want to make some initial analysis on the structure of graph itself.
In particular we want to figure out:
 <ul>
 <li>  If the graph is directed or not;
</li>
    <li>
    The number of edges;
 </li>
    <li>
    The number of nodes;
    </li>
 </li>
    <li>
    The average node degree;
    </li>
 </li>
    <li>
    The density of the graph. 
    </li>
</ul>
</span>

#### Directed or Undirected
Two approaches are proposed to approve the graph is directed.

In [43]:
graph['52']

['401135', '1069112', '1163551']

In [44]:
print(graph['1069112'])

['1060396', '1061304', '1062611', '1066969', '1069008', '1069113', '1069258', '1069275', '1656982']


<span style="font-family: Babas; font-size: 1.2em;">
The graph is not undirected because the node 52 has edge with 1099112 and the reverse is not true
<br>
The second reason for demonstrating that the graph is directed is that the number of keys in relation to the number of nodes that we computed below is not the same: this means that there are nodes without any edge that comes back, otherwise we could find these nodes also in the dictionary graph as key.  
</span>

#### Edges

<span style="font-family: Babas; font-size: 1.2em;">
Below we calculate the number of the edges of the graph, As we created a dictionary in which the keys of the dictionary shows the Graph nodes and the values of that keys are the list of the nodes that the key(this particular node ) is connected, Therefore , we calculated the len of list of each key. As the result we calculate the number of edges.
</span>

In [45]:
#We create an empy list of edges that we have to populate each time
edges = []
for e in graph:
    a = graph[e]
     #a is a list of nodes that are linked to the node e
    for i in range(len(a)):
        edges.append(a[i])

In [46]:
len(edges) 

2645247

#### The graph is composed by 2645247 edges

In [47]:
len(graph.keys())

428957

#### Nodes

<span style="font-family: Babas; font-size: 1.2em;">
Below the nodes of the graph are calculated, to do so , we read the file line by line  each line consists of two elements.The first Element refers to node, and the second element refers to the node that is connected to the first. All the nodes are appended to the list. Then the set is used to find the nodes and avoiding the duplication.
</span>

In [48]:
#We create an empty list of nodes that we have to populate each time nodes = []
nodes = []
with open("wiki-topcats-reduced.txt") as f:
        for line in f:
            #The two elements of each splitted line are nodes
            line = line.strip().split("\t")
            nodes.append(line[0])
            nodes.append(line[1])
nodes=set(nodes) #we used the set function to avoid duplicates

In [49]:
#The Number of Nodes
len(nodes) 

461193

#### The graph is composed by 461193 nodes

In [52]:
#for take the node with the maximum number of edges:
m = 0
node = ''
for e in graph:
    l=graph[e]
    if len(l) > m:
        m = len(l)
        node=e

In [56]:
node, m  

('1061816', 1983)

In [64]:
c = 0 # count how many isolated nodes there are
for node in G:
    if len(G.edges(node)) == 0:
        c+=1

In [65]:
c

32236

#### Average Node

<span style="font-family: Babas; font-size: 1.2em;">
The average node degree is given by the number of edges over the number of nodes, In order to calculate that ,
from graph dictionary that we created , the keys of the graph represent the number of node,Therefore the length of the
graph keys are calculated and number of edges in the dictionary are equal to the length of the edges of each node.
</span>

In [12]:
#Average Node Degree
count = 0
for lst in graph.values():
    count += len(lst)
    #graph.values() returns the lists of nodes which are linked to each key
avgNodeDegree = count/len(graph.keys())

In [33]:
avgNodeDegree

5.735661642739591


<span style="font-family: Babas; font-size: 1.2em;">
We have 6 edges distributed for each node, but we shown that, for example, the maximum number of edges in the graph is close to be 2000.
And, at the same time, there are 32236 nodes isolated.
Thus, we got this average.
</span>

#### Density

<span style="font-family: Babas; font-size: 1.2em;">
The Dense in calculated with following formula; E is the number of edges and V is the number of Nodes , As we calculated the Edges and Nodes before Just it is applied in the following Formula and we got the Result.
<br>
D=2|E|/(|V|(|V|-1))
<br>
The Density is between 0 and 1 the closer to 1 the more dense.As the result that is obtained is much closer to 0 , it shows that It is not a Dense Graph.
</span>

In [14]:
#Density
dense = len(edges)/(len(nodes)*(len(nodes)-1))
dense

1.2436602635647606e-05

## Distance-Rank

<span style="font-family: Babas; font-size: 1.2em;">
For the computation of the shortest paths, we needed two structure:
    the graph and the categories with all their nodes associated.
 <br>

 In the following Script, by the create_graph_and_dict function the Graph is created , in that function DiGraph() from networkx is used in order to create the Direted Graph. The category dictionary is also cretaed. The category dictionary is the cleaned file of "wiki-topcats-categories.txt". As there were some nodes in each category that they were not existed in the graph nodes,because it was reduced, those nodes were deleted. At the end a category dictionary created that it contains only the nodes that are existed. Moreover, the Categories that has the length less than 3500 are deleted(as the question asked).
</span> 

In [2]:
G, category_dict=f.create_graph_and_dict()
category_list = list(category_dict.keys())
input_category='Indian_films'

#### Creating the input parameter for the distance _graph() function

<span style="font-family: Babas; font-size: 1.2em;">
Below two lists are created for the input of BFS(which we called here graph-distance() function) algorithm. As we had 35 input , two lists that each of them contains, 16 categories are created; Because as it will be mention in the thread part; each time we run for 16 categories and the input of the distance function(which will be explain) is four categories.Each element of the lst contains four categories which is one of the parameter of the distance function. As the categories are 35 and we wanted to avoid Memory Error (which this part is explained completely in the thread part); for the last 3 elements that will be remain, we runned separatly on another function, which is distance_graph2 function. The lst is printed below.
</span> 

In [4]:
# Below created the list of lists for the first 16 categories 
auxiliary_list = list(category_dict.keys())
auxiliary_list.remove(input_category)
i=0
lst=[]
while i < 16:
    lst.append(auxiliary_list[i:i+4])
    i+=4

#this is the next list
lst_2=[]
while i<32:
    lst_2.append(auxiliary_list[i:i+4])
    i+=4

In [5]:
lst

[['English_footballers',
  'The_Football_League_players',
  'Association_football_forwards',
  'Association_football_goalkeepers'],
 ['Association_football_midfielders',
  'Association_football_defenders',
  'Living_people',
  'Year_of_birth_unknown'],
 ['Harvard_University_alumni',
  'Major_League_Baseball_pitchers',
  'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
  'Year_of_death_missing'],
 ['English_cricketers',
  'Year_of_birth_missing_(living_people)',
  'Rivers_of_Romania',
  'Main_Belt_asteroids']]


#### Threads

<span style="font-family: Babas; font-size: 1.2em;">

Below multi-threading is used to facilitate the execution of the BFS algorithm, in order to find the distances between the category that we chosen and the others. 
As the input was too large for BFS function . Multithreading has been used for running the same method for different categories at the same time, in order to obtain fast result instead of run each time the method for one category. Four threads are created to access to the BFS function.  Each time that the function is called it computes four categories (we decided this number for avoiding the Memory Error and to overload too much each thread.) The same for the procedure used for calling threads: we decided to call four thread per procedure for the same reason. 

res list was the resources that keep all the output of threads. As soon as the thread finished, the res list has been updated.
</span> 


<span style="font-family: Babas; font-size: 1.2em;">
After getting the informations needed, we finally computed the method "distance_graph" inside threads
</span>

In [5]:
res_list = []
def threadGraph(lst):
    res_list.append(f.distance_graph(G,input_category, lst,category_dict))

threads = []
#four threads are created here
for a in range(4):
    t = threading.Thread(name = a, target = threadGraph,
                       args = [lst[a]])
    
    threads.append(t)
    t.start()
Graphdf=pd.DataFrame()
for i in res_list1:
    Graphdf1=pd.DataFrame(i)
    Graphdf=Graphdf.append(Graphdf1)
Graphdf.to_csv("Graph1.csv",index=False)

#### The Following dataframe is the out put of the first four threads that we runned.

In [36]:
Graphdf1

,0,1
0,English-language_films,6.000000
1,American_films,inf
2,Fellows_of_the_Royal_Society,8.000000
3,People_from_New_York_City,7.000000
0,Black-and-white_films,7.000000
1,Year_of_birth_missing,inf
2,Place_of_birth_missing_(living_people),8.000000
3,Article_Feedback_Pilot,6.000000
0,Asteroids_named_for_people,inf
1,English-language_albums,7.000000


In [16]:
res_list1 = []
def threadGraph1(lst):
    res_list1.append(f.distance_graph(G,input_category, lst,category_dict))

threads = []
for a in range(4):
    t = threading.Thread(name = a, target = threadGraph1,
                       args = [lst_2[a]])    
    threads.append(t)
    t.start()
Graphdf=pd.DataFrame()
for i in res_list1:
    Graphdf1=pd.DataFrame(i)
    Graphdf=Graphdf.append(Graphdf1)
Graphdf.to_csv("Graph2.csv",index=False)

#### Below the result of the function is printed. This result is saved in Graph2.csv as a dataframe

In [14]:
#the result is like below for each category the median shortest path
print(res_list1)

[[('English-language_films', 6.0),('American_films', inf),('Fellows_of_the_Royal_Society', 8.0),('People_from_New_York_City', 7.0)],[('Black-and-white_films', 7.0),('Year_of_birth_missing', inf),('Place_of_birth_missing_(living_people)', 8.0),('Article_Feedback_Pilot', 6.0)],[('Asteroids_named_for_people', inf),('English-language_albums', 7.0),('English_television_actors', 6.0),('British_films', 6.0)],[('American_Jews', 7.0),('American_television_actors', 6.0),('American_film_actors', inf),('Debut_albums', 7.0)]]


#### the distance_graph2 is executed for the last three remaining Categories

In [15]:
d=f.distance_graph2(G,input_category,['American_military_personnel_of_World_War_II', 'Windows_games','Indian_films'],category_dict)
Graphdf=pd.DataFrame()
Graphdf=Graphdf.append(d)
Graphdf.to_csv("Graph3.csv",index=False)

<span style="font-family: Babas; font-size: 1.2em;">
 <ul>
 <li>  We runned the previous algorithm that contains the Search of the Shortest Paths using threads for three times.
 This was due to the fact that data were big.
 Input given to the method: graph, input category, list with categories in order to find the shortest paths. 
</li>
    <li>
    Because we runned three times in different moments, we saved three lists with the results in this format:
 (Name of the Category, Rank with the input category with this category);
 </li>
    <li>
 Thus, we merged all the categories with the ranks together.
Below there is the Dataframe related to categories, sorted from the closest one until the furthest one by distance in order to rank categories.
    </li>
</ul>
</span>

<span style="font-family: Babas; font-size: 1.2em;">
Working on the merging of all results obtained
</span>

In [8]:
dfg=pd.read_csv('Graphdf1.csv')
dfg1 = pd.read_csv('Graphdf2.csv')
dfg2 = pd.read_csv('Graph3.csv')
dfg=dfg.reset_index(drop=True)
dfg = dfg.rename(columns={"0": "Category", "1": "Distance"})
dfg = dfg.sort_values(by='Distance')
dfg = dfg.reset_index(drop=True)

In [10]:
dfg.to_csv('ranking_table.csv', index = False)

In [11]:
dfg=pd.read_csv('ranking_table.csv')

<span style="font-family: Babas; font-size: 1.2em;">
List of ranks cleaned
</span>

In [13]:
dfg

,Category,Distance
0,Indian_films,0.000000
1,American_television_actors,6.000000
2,British_films,6.000000
3,English_television_actors,6.000000
4,Article_Feedback_Pilot,6.000000
5,English-language_films,6.000000
6,Debut_albums,7.000000
7,American_Jews,7.000000
8,English-language_albums,7.000000
9,Black-and-white_films,7.000000


<span style="font-family: Babas; font-size: 1.2em;">

Last point: We want to keep an information, score, into each node. First, we initialized every node as a negative value, in order to make a check (if we still didn't pass throught that node it will be still a negative number, but if there are no in edges this information will be 0). 

This initialization will be done into G, our original graph.

</span>

In [17]:
G = f.steps(G,category_dict)

<span style="font-family: Babas; font-size: 1.2em;">

By the following Script the category and the Id of the Node plus the Score of the Node is extracted from the Graph G, as it is 
explained in the function file , except the Id of the Node the category and Score are the attributes of each Node. At the end we obtained a list of dictionaries, each dictionary contains the edge Id, score and category.The numba library is used to facilitate the operation, because it converts it to C code. For that reason it runs faster.
</span>

In [19]:
list1 = []
@autojit
def graphscore(e):
    score_dict={}
    score_dict['Node']=e
    score_dict['Score']=G.node[e]['score']
    score_dict['Category']=G.node[e]['Category']

    return(score_dict)
for e in G:
    list1.append(graphscore(e))

####  In the following script the list of dictionaries are converted to dataframe. Then the dataframe is sorted by the Score column.

In [5]:
dfscore=pd.DataFrame(list1)
dfscore = dfscore.sort_values(by='Score', ascending=False)
dfscore.head(100)

,Category,Node,Score
10,American_film_actors,1400636,412930200
18666,American_film_actors,1025352,217343537
3057,American_film_actors,1061501,116923346
1931,American_film_actors,1181827,113641231
50420,American_film_actors,1026878,98591170
56137,American_film_actors,1061469,93403894
58437,American_film_actors,1061487,83633468
4653,American_film_actors,1163630,60086906
13232,American_film_actors,1265399,58954983
67507,Year_of_birth_unknown,168194,56899513


<span style="font-family: Babas; font-size: 1.2em;">
The dataframe that we created above is group by the Category; categories are sorted in relation of the rank ,  and scores for each node in its category are sorted in descending mode. Below the first Five highest score for each category is shown.
</span>

In [10]:
df_sort_score_in_cat=pd.read_csv("df_sort_score_in_cat.csv")
#dfname=pd.read_csv("wiki-topcats-page-names.txt")

d = {}
with open('wiki-topcats-page-names.txt', 'r') as f:
    for line in f:
        line=line.split(' ', 1)
        d[str(line[0])] = line[1].strip()
df_sort_score_in_cat['Name'] = df_sort_score_in_cat['Node'].map(lambda x: d[str(x)])
category_list.remove('Indian_films')
df_sort_score_in_cat.groupby('Category').head(5)

,Category,Node,Score,Name
0,American_Jews,1061960,954351,Steven Spielberg
1,American_Jews,1062582,550629,Gene Siskel
2,American_Jews,1061552,536242,William Wyler
3,American_Jews,1061631,483951,Pauline Kael
4,American_Jews,1062226,345197,Jack Warner
2683,American_film_actors,1400636,412930200,Ronald Reagan
2684,American_film_actors,1025352,217343537,Frank Sinatra
2685,American_film_actors,1061501,116923346,John Wayne
2686,American_film_actors,1181827,113641231,Elvis Presley
2687,American_film_actors,1026878,98591170,Judy Garland
